In [1]:
import transformers
import pprint

# §1 Transformers库使用演示

情感分析(sentiment-analysis)

In [2]:
classfier = transformers.pipeline(
    task="sentiment-analysis", 
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english", 
    device="cuda:0"
)

pprint.pprint({
    "单句": classfier("I hate you!"),
    "多句": classfier(["I love you", "Fuck you"])
})

d:\.conda\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'单句': [{'label': 'NEGATIVE', 'score': 0.9987472295761108}],
 '多句': [{'label': 'POSITIVE', 'score': 0.9998656511306763},
        {'label': 'NEGATIVE', 'score': 0.9923542737960815}]}


零样本分类(zero-shot-classification)

In [3]:
classfier = transformers.pipeline(
    task="zero-shot-classification",
    model="facebook/bart-large-mnli",
    device="cuda:0"
)
pprint.pprint(classfier(
    "I attended a meeting today.",
    candidate_labels=["diary", "news", "finance"],
    clean_up_tokenization_space=False # 保证tokenization的正向/逆向运算一致性
))

{'labels': ['news', 'diary', 'finance'],
 'scores': [0.7021396160125732, 0.20275545120239258, 0.09510485827922821],
 'sequence': 'I attended a meeting today.'}


d:\.conda\Lib\site-packages\transformers\models\bart\modeling_bart.py:496: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


文本生成(text-generation)

In [4]:
generator = transformers.pipeline(
    task="text-generation",
    model="openai-community/gpt2",
    device="cuda:0",
)
pprint.pprint(generator(
    "I love you, such as",
    truncation=True, max_length=20,
    num_return_sequences=5,
))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I love you, such as I can!"\n'
                    '\n'
                    'Maggie and her friend were out in'},
 {'generated_text': 'I love you, such as I am.\n'
                    '\n'
                    'I am so very sorry. I must make'},
 {'generated_text': "I love you, such as I did, but I don't get it. It's my "
                    'problem'},
 {'generated_text': 'I love you, such as I did.\n'
                    '\n'
                    'You\'ll feel great in there," the queen'},
 {'generated_text': 'I love you, such as it is." I\'m so grateful to our '
                    'children for helping them grow'}]


掩码填充(fill-mask)

In [5]:
unmasker = transformers.pipeline(
    task="fill-mask",
    model="distilbert/distilroberta-base",
    device="cuda:0",
)
pprint.pprint(unmasker(
    "This is a <mask> fruit, which tastes good."
))

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.08490116894245148,
  'sequence': 'This is a ripe fruit, which tastes good.',
  'token': 23318,
  'token_str': ' ripe'},
 {'score': 0.06573793292045593,
  'sequence': 'This is a tropical fruit, which tastes good.',
  'token': 10602,
  'token_str': ' tropical'},
 {'score': 0.06305697560310364,
  'sequence': 'This is a dried fruit, which tastes good.',
  'token': 16380,
  'token_str': ' dried'},
 {'score': 0.04835207387804985,
  'sequence': 'This is a fresh fruit, which tastes good.',
  'token': 2310,
  'token_str': ' fresh'},
 {'score': 0.047243986278772354,
  'sequence': 'This is a juicy fruit, which tastes good.',
  'token': 28068,
  'token_str': ' juicy'}]


命名实体识别(NER, named entity recognition)

In [6]:
ner = transformers.pipeline(
    task="ner",
    model="dbmdz/bert-large-cased-finetuned-conll03-english",
    grouped_entities=True, # 多个Token可以共同构成一个Entity
    device="cuda:0"
)
pprint.pprint(ner(
    "My name is Sylvain and I work at Hugging Face in Brooklyn."
))
# PER: Person
# ORG: Organization
# LOC: Location

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
d:\.conda\Lib\site-packages\transformers\pipelines\token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


[{'end': 18,
  'entity_group': 'PER',
  'score': 0.9981694,
  'start': 11,
  'word': 'Sylvain'},
 {'end': 45,
  'entity_group': 'ORG',
  'score': 0.9796019,
  'start': 33,
  'word': 'Hugging Face'},
 {'end': 57,
  'entity_group': 'LOC',
  'score': 0.9932106,
  'start': 49,
  'word': 'Brooklyn'}]


问答系统(question-answering)

In [7]:
question_answerer = transformers.pipeline(
    task="question-answering",
    model="distilbert/distilbert-base-cased-distilled-squad",
    device="cuda:0"
)
pprint.pprint(question_answerer(
    {
        "question": "Who are you?",
        "context": "You are a lovely little boy."
    }
))

{'answer': 'a lovely little boy',
 'end': 27,
 'score': 0.6046934723854065,
 'start': 8}


文本摘要(summarization)

In [8]:
summarizer = transformers.pipeline(
    task="summarization",
    model="sshleifer/distilbart-cnn-12-6",
    device="cuda:0"
)
pprint.pprint(summarizer(
    "Rapidly developing economies such as China and India, as well as other industrial countries in Europe and Asia, continue to encourage and advance the teaching of engineering.",
    max_length=30, min_length = 20
))

[{'summary_text': ' Rapidly developing economies such as China and India, as '
                  'well as other industrial countries in Europe and Asia, '
                  'continue to encourage and advance the'}]


翻译(translation)

In [9]:
translator = transformers.pipeline(
    task="translation",
    model="Helsinki-NLP/opus-mt-zh-en",
    device="cuda:0"
)
pprint.pprint(translator(
    "你好"
))

d:\.conda\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Hello.'}]
